In [1]:
import os
from pathlib import Path
base = Path().cwd()
# switch to home directory to import helper scripts
if str(base).split('/')[-1] != 'runtime-monitoring':
    DATASET = str(base).split('/')[5]
    os.chdir('../../..')
    base = Path().cwd()

print(DATASET)
print(base)

MNIST
/home/ah19/runtime-monitoring


In [2]:
# import env virables
from dotenv import load_dotenv
load_dotenv(base / 'configurations' / 'thresholds.env')

# import helper functions
from utilities.pathManager import fetchPaths
from utilities.bdd_threshold import build_threshold

In [3]:
# env variables
PREFIX = 'Adam-256-30'
FILENAME_POSTFIX = f'{DATASET}_{PREFIX}'
FALVOR = 'raw'

In [4]:
paths = fetchPaths(base, DATASET, PREFIX)

path_data = paths['data']
path_model = paths['model']
path_saved_models = paths['saved_models']
path_bdd = paths['bdd'] / FALVOR
path_lhl = paths['lhl_' + FALVOR]

In [6]:
import pandas as pd
import numpy as np


train = pd.read_csv(path_bdd / 'relu-neurons_bdd_train_CUDD.csv')
test = pd.read_csv(path_bdd / 'relu-neurons_bdd_test_CUDD.csv')

In [8]:
train.shape

(60000, 26)

In [7]:
train.head()

,x1,x2,x3,x5,x6,x7,x9,x11,x12,x13,...,x24,x25,x26,x27,x28,x29,y,true,bdd_0,bdd_1
0,1,0,1,0,0,0,1,1,0,0,...,0,0,1,0,0,0,5,True,1,1
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,3,True,1,1
2,1,1,0,1,0,0,0,0,0,1,...,1,1,1,0,0,0,1,True,1,1
3,1,0,0,1,0,0,1,0,0,0,...,1,1,1,0,0,0,1,True,1,1
4,0,1,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,True,1,1


In [69]:
def score_dataframe(df, bdd_col='bdd'):
    """TODO"""
    # return empty df if false column name was passed
    if bdd_col not in df.columns:
        return pd.DataFrame({
                    'y': []
                    ,'count': []
                    ,'_false': []
                    ,'_false_misclassified': []
                    ,'outOfPattern': []
                    ,'outOfPatternMisclassified': []
                    ,'eta':[]
                })
    
    # how many instances per class
    df_all_classes = df[['y', 'true']].groupby('y').count().sort_index()
    df_all_classes.columns = ['count']
    
    # how many patterns not found
    df_out_of_pattern_images = df.loc[df[bdd_col] == 0, ['y', bdd_col]].groupby('y').count().sort_index()
    df_out_of_pattern_images.columns = [bdd_col + '_false']

    # how many patterns not found and misclassified
    df_out_of_pattern_misclassified_images = df.loc[(df[bdd_col] == 0) & (df['true'] == False), ['y', bdd_col]].groupby('y').count().sort_index()
    df_out_of_pattern_misclassified_images.columns = [bdd_col + '_false_misclassified']
    
    df_scores = df_all_classes.join(df_out_of_pattern_images).join(df_out_of_pattern_misclassified_images)
    
    del df_out_of_pattern_images, df_out_of_pattern_misclassified_images

    
    total_images = df_all_classes['count'].sum()
    out_of_pattern_images = (df[bdd_col] == 0).sum()
    out_of_pattern_misclassified_images = ((df['true'] == False) & (df[bdd_col] == 0)).sum()
    df_scores.loc['all', :] = [total_images, out_of_pattern_images, out_of_pattern_misclassified_images]
    
    # if data frame return 0 rows, a nan will be placed
    df_scores.fillna(0, inplace=True)

    # calculate metrics
    df_scores['outOfPattern'] = df_scores[bdd_col + '_false'] / df_scores['count']
    
    # more misclassfied and undetected pattern means the monitor is correctly detecting unfamiliar patterns
    df_scores['outOfPatternMisclassified'] = df_scores[bdd_col + '_false_misclassified'] / df_scores[bdd_col + '_false']

    
    # add mean of all classes
    a1 = df_scores.loc[df_scores.index != 'all', 'outOfPattern'].mean()
    a2 = df_scores.loc[df_scores.index != 'all', 'outOfPatternMisclassified'].mean()

    # if class is never Misclassified and bdd recognize all of its patterns
    # both outOfPattern and outOfPatternMisclassified will be 0
    # so the division will result in NaN, thus will be replaced by zero
    # because we don't know how the monitor will react once the class's data start to get outdated
    df_scores['outOfPatternMisclassified'].replace({np.nan:0.0, 0.0:1.0}, inplace=True)
    df_scores['outOfPattern'].replace({np.nan:0.0}, inplace=True)
    
    # no missclassification for a class
    df_scores[bdd_col + '_false'].replace({np.nan:0.0}, inplace=True)
    df_scores[bdd_col + '_false_misclassified'].replace({np.nan:0.0}, inplace=True)

    # how many pattern were undetected but correctly classified
    df_scores[bdd_col + '_false_classified'] = df_scores[bdd_col + '_false'] - df_scores[bdd_col + '_false_misclassified']
    df_scores['outOfPatternClassified'] = 1 - df_scores['outOfPatternMisclassified']
    
    out_of_pattern_classified_images = out_of_pattern_images - out_of_pattern_misclassified_images
    df_scores.loc['all', bdd_col + '_false_classified'] = out_of_pattern_classified_images
    # df_scores['outOfPatternClassified'].replace({np.nan:0.0, 0.0:1.0}, inplace=True)
    a3 = df_scores.loc[df_scores.index != 'all', 'outOfPatternClassified'].mean()
    df_scores.loc['all_mean', :] = [total_images, out_of_pattern_images, out_of_pattern_misclassified_images, 0, a1, a2, a3]
    
    # reorder columns
    df_scores = df_scores[['count', bdd_col + '_false', bdd_col + '_false_misclassified', bdd_col + '_false_classified',
                        'outOfPattern','outOfPatternMisclassified','outOfPatternClassified']]

    if bdd_col=='bdd':
        return df_scores.reset_index()
    return df_scores


t = score_dataframe(test, 'bdd_0')
t

,count,bdd_0_false,bdd_0_false_misclassified,bdd_0_false_classified,outOfPattern,outOfPatternMisclassified,outOfPatternClassified
y,,,,,,,
0,980.0,0.0,0.0,0.0,0.000000,0.000000,1.0
1,1135.0,0.0,0.0,0.0,0.000000,0.000000,1.0
2,1032.0,2.0,0.0,2.0,0.001938,1.000000,0.0
3,1010.0,0.0,0.0,0.0,0.000000,0.000000,1.0
4,982.0,3.0,0.0,3.0,0.003055,1.000000,0.0
5,892.0,4.0,0.0,4.0,0.004484,1.000000,0.0
6,958.0,2.0,0.0,2.0,0.002088,1.000000,0.0
7,1028.0,0.0,0.0,0.0,0.000000,0.000000,1.0
8,974.0,6.0,0.0,6.0,0.006160,1.000000,0.0
